In [18]:
import pandas as pd
from ddr import *
pd.options.mode.copy_on_write = True

In [83]:
# kolommen controleren controleren met df.columns zonder 'usecols='
vl = pd.read_csv('vrlijst_lijstdef.csv',
                  usecols=[0, 1, 2, 5, 8, 12, 16, 19],
                  dtype={'VERVALLEN': 'boolean', 'DATUM': 'string'})

In [86]:
vl.loc[vl['DATUM'].str.contains('2025')]

,LIJSTID,CATEGORIE,LIJSTNAAM,DATUM,VERVALLEN,KOPPID,VRLIJSTTYPEID,SOORTID
4102,CS00065063,CAR,PROM: Cardiologie Follow up,2025-02-18 00:00:00,False,CS00015801,NaN,S
4103,CS00065070,CAR,PROM: Cardiologie (PCI) - Electieve patient Ba...,2025-02-18 00:00:00,False,CS00015808,NaN,S
32418,CS00065047,SCORE,Seattle Angina Questionnaire (SAQ-7),2025-02-05 00:00:00,False,CS00015789,NaN,S
33702,1000019817,LABKCL,Studie: UNICIT SubjectNr,2025-03-18 00:00:00,False,2000004508,NaN,S
41280,1000019820,HEMA,Conditioneringsschema Plan,2025-03-20 00:00:00,False,2000004509,NaN,S
...,...,...,...,...,...,...,...,...
61214,CS00065113,MDO,MDO Aanmelding,2025-02-19 00:00:00,False,NaN,NaN,L
61215,CS00065117,SCORE,RIS Inclusiecriteria 6MWT,2025-02-19 00:00:00,False,NaN,NaN,NaN
61216,1000019848,MDO,Actuele status patient,2025-03-25 00:00:00,False,NaN,NaN,NaN
61217,1000019850,MDO,Toelichting actuele status patiënt,2025-03-25 00:00:00,False,NaN,NaN,NaN


In [28]:
vr = pd.read_csv('vrlijst_vragen.csv', low_memory=False,
                usecols=[0, 1, 3, 6, 7, 8, 12, 27, 33, 39, 40],
                dtype={'MAGLEEG': 'boolean', 'VERVALLEN': 'boolean', 'ENKELV': 'boolean',
                       'CONTROLID': 'Int64'})

In [66]:
vc = pd.read_csv('vrlijst_controls.csv',
                usecols=[0, 1, 4])

In [61]:
tl = pd.read_csv('vrlijst_treelay.csv',
                 usecols=[0, 1, 2, 3, 4, 8, 9, 12, 13],
                 dtype={'VOLGNR':'Int64', 'HERHAALBAA': 'boolean', 'ONZZANTW': 'boolean', 'NEWPAGE': 'boolean'})

In [89]:
start = Lijst.make_one(vl.loc[vl['LIJSTID'] == 'CS00065113'])

In [77]:
def vraag_type(v):
    descr = vc.loc[vc['CONTROLID']==v.CONTROLID].to_dict('records')
    return descr[0]['NAAM']

In [75]:
def add_children(vlnode, distance=0):
    tlnodes = tl.loc[tl['LIJSTID'] == vlnode.LIJSTID].copy()
    tlnodes.sort_values(by='VOLGNR', inplace=True)
    for node in tlnodes.to_dict('records'):
        childid = node['CHILDID']
        indent = '  ' * distance
        onz = node['ONZZANTW']
        if node['TYPE'] == 'V':
            vraag = Vraag.make_one(vr.loc[vr['VRAAGID'] == childid])
            vlnode.children.append(vraag)
            print(f"{indent}{node['VOLGNR']} V: {vraag} type={vraag_type(vraag)} verberg={onz}")
        elif node['TYPE'] == 'L':
            lijst = Lijst.make_one(vl.loc[vl['LIJSTID'] == childid])
            add_children(lijst, distance+1)
            vlnode.children.append(lijst)
            print(f"{indent}{node['VOLGNR']} L: {lijst} verberg={onz}")
        else:
            print(f"{indent}onbekend type {node['TYPE']}")

In [90]:
add_children(start)

0 V: Reden van aanmelding MDO (CS00276754 vervallen=False) type=RTF Invoer Memo verberg=False
1 V: Datum van aanmelding (CS00276755 vervallen=False) type=Datum verberg=False
2 V: Opmerkingen bij MDO (CS00276756 vervallen=False) type=Memo verberg=False
3 V: Patiënt eerder besproken (CS00214848 vervallen=False) type=Keuzeselectiegroep verberg=True
